# Exploration of OpenStreetMap data for Philadelphia, Pennsylvania.
Kasey Cox, May 2017

OSM XML file from MapZen.com Metro Extracts:
https://mapzen.com/data/metro-extracts/metro/philadelphia_pennsylvania/101718083/Philadelphia/

In [1]:
%autosave 0

Autosave disabled


## Auditing

Using __"audit.py"__ and __"audit2.py"__, I checked out the structure and contents of the data. The majority of problems were found in the tag attributes for the nodes, ways, and relations.

### Tag "k" attributes

Below gives the count of "k" attributes falling into predefined format categories.

__`audit_k_formats(osmfile)` in "audit.py":__
> `{'problemchars': 9, 'other': 182007, 'single_colon': 660267, 'lett_undrsc': 951883}`
>
> __'lett_undrsc'__ contains "k" attributes that only have lowercase letters and/or underscores.
>
> __'single_colon'__ contains "k" attributes with only lowercase letter and/or underscores on each side of a single colon
>
> __'problemchars'__ contains "k" attributes with problematic, special charaters like ?, quotes, or brackets.
>
> __'other'__ contains "k" attributes that do not fit into the above categories

Here are some examples for 'problemchars' and 'other':
>  __problemchars (all):__
>
> `set(['Price and Price Elder Law', 'tiger.source:tlid', 'fuel:2.14', 'max speed', 'store number', 'service area'])`
> 
> __other (a few examples):__
>
> `set(['gnis:ST_alpha', 'gnis:County_num', 'gnis:ST_num', 'FIXME', 'gnis:Class', 'gnis:County', 'TODO'])`

### Tag "v" attributes

Below are the results from taking 2 different "k" values and using `audit_v_format(osmfile, kval)`. These results demonstrate the different formatting for the "v" values of the given "k"s.

__`audit_v_format(osmfile, kval)`__ (in "audit2.py") (whilst limiting to the first 100,000 lines):
> `'k'='addr:street'`
>
> `{'problemchars': 0, 'other': 0, 'whitespaces': 105, 'lett_undrsc': 0}`

> `'k'='addr:postcode'`
>
> `{'problemchars': 0, 'other': 79, 'whitespaces': 0, 'lett_undrsc': 1}`
>
> - `'lett_undrsc'` contains any string with only letters and/or underscores
> - `'whitespaces'` contains any string with whitespace characters
> - `'problemchars'` contains any string with problematic characters like semi-colons, question marks, etc.
> - `'other'` contains any string that doesn't fit into the above mentioned categories

Much of the variation comes from (undesirable) inconcistencies and invalid data entry. This is demonstrated by exploring the data with __`audit_kv_pairs(osmfile, kval)`__ (in "audit2.py").

`k='addr:street'`
- inclusion of non-street address information
    - South Clinton Avenue Ste. 111
- abbreviations with inconsistent use of periods
    - E. Mt Airy Ave
    - Cooper St
    - Wistar Rd.
    - S 41st St
    
`k='addr:postcode'`
- some postcodes have additional 4 digits
    - 18940
    - 19030-4005
- invalid values
    - PA
    - '601 Stokes Road Medford, NJ 08055'
    - 'NJ 08083'

### Audit impressions so far
Seeing the above makes it clear that there could be innumerable inconsistencies in such a large data set. I could spend a lot of time (days, weeks?) finding every inconsistency in every "k" attribute and "v" attribute. 

Additional subelements exist for ways (nd) and relations (member) that likely have the same problems as tags.

It makes sense then to choose which "k" attributes are most valuable to my analysis and move forward with only those for cleaning and importing as a JSON file into Mongo DB.

__I am going to stick to nodes for cleaning, exporting as a JSON file, and importing into MongoDB.__

## ----------------------------------------------------------
## Cleaning, Exporting, Importing to MongoDB

Prepared data for export as JSON file in the following format:
```
example_format =
{
    "id": "2406124091",
    "type": "node",
    "created": {"version":"2",
               "changeset":"17206049",
               "timestamp":"2013-08-03T16:43:42Z",
               "user":"linuxUser16",
               "uid":"1219059"},
    "pos": [41.9757030, -87.6921867],
    "address": {"housenumber":"5157",
               "postcode":"60625",
               "street":"North Lincoln Avenue"},
    "amenity": "restaurant",
    "cuisine": "Mexican",
    "name": "La Cabana De Don Luis",
    "phone": "1 (773)-271-5176"
}
```

The code in __"clean.py"__ contains all of the cleaning procedures performed on the data. This file also writes the cleaned information to a JSON file.

Here is an overview of the cleaning performed:
- All address components are checked for validity. If a value is not appropriate (e.g. "PA" in postalcode), "Invalid" is given.
- Street prefixes are fixed ("N" becomes "North"; "Mt" becomes "Mount")
- Street suffixes are fixed ("St." or "St" becomes "Street")
- The latitude and longitude are converted to floats and stored as a list in the key "pos".
    
I imported my cleaned JSON file into MongoDB using the mongoimport command in the terminal:
> `mongoimport -d test -c osmdata --file philadelphia_pennsylvania.osm.json`

## ----------------------------------------------------------
## Queries

```
> db.osmdata.stats()
{
	...,
	"count" : 3113088,
	"size" : 752185600,
	"avgObjSize" : 241,
	"numExtents" : 17,
	"storageSize" : 857440256,
	"lastExtentSize" : 227803136,
	...
}
```

__Number of entries in collection__: 3,113,088 (since I only put node elements into the JSON file)


Full code for the below queries is contained in __"query.py"__.

### NUMBER OF UNIQUE USERS
```
user_query = [  {"$group": {"_id":"$created.uid", "unique_users": {"$addToSet":"$_id"}}},
                {"$project":{"_id":0, "total_users":{"$size":"$unique_users"}}},
                {"$sort":{"total_users":-1}},
                {"$limit":1}
             ]
```
__Results:__ 
> `{u'total_users': 764695}`

### NUMBER OF SCHOOLS IN PHILADELPHIA

After creating a text index:
```
school_query = [    {"$match":{"$text":{"$search":"School"}}},
                    {"$group":{"_id":"$address.city", "count":{"$sum":1}}},
                    {"$match": {"$or":[{"_id":"Philadelphia"}, {"_id":"philadelphia"}, {"_id":None}] }}
               ]

academy_query = [   {"$match":{"$text":{"$search":"Academy"}}},
                    {"$group":{"_id":"$address.city", "count":{"$sum":1}}},
                    {"$match": {"$or":[{"_id":"Philadelphia"}, {"_id":"philadelphia"}, {"_id":None}] }}
                ]
```

__Results:__
```
Entries with "School"
{u'count': 8, u'_id': u'Philadelphia'}
{u'count': 1456, u'_id': None}
{u'count': 1, u'_id': u'philadelphia'}

Entries with "Academy"
{u'count': 2, u'_id': u'Philadelphia'}
{u'count': 106, u'_id': None}
```

__Comment:__
> There are potentially Philadelphia schools in results that do not have a city tagged to it (None), too.

### MOST POPULAR CUISINE
```
cuisine_query = [   {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
                    {"$project":{"_id":0, "name":"$_id", "count":"$count"}},
                    {"$sort":{"count":-1}},
                    {"$limit":10}
                ]
```

__Result:__
```
{u'count': 3112260, u'name': None}
{u'count': 155, u'name': u'pizza'}
{u'count': 66, u'name': u'burger'}
{u'count': 64, u'name': u'chinese'}
{u'count': 62, u'name': u'coffee_shop'}
{u'count': 57, u'name': u'italian'}
{u'count': 54, u'name': u'sandwich'}
...
```

__Comment:__
Pizza is a type of Italian food. "italian" has 57 counts and "pizza" has 155. Italian cuisine comfortably leads the popularity list with 212 counts.


### RESTAURANTS NEAREST TO THE AIRPORT:
After creating a 2D index:
```
pos_query = [   {"$geoNear": {"near":[39.871944, -75.241111], "distanceField":"dist.calculated", 
                    "query": {"amenity": {"$in": ["restaurant", "bar", "pub", "fast_food", "bar;pub"] } }}},
                {"$limit":1}
            ]
```

__Result:__
> {u'amenity': u'fast_food', u'dist': {u'calculated': 0.004474988617863642}, u'name': u"Green Leaf's", u'created': {u'changeset': u'34171919', u'version': u'1', u'user': u'dbaron', u'timestamp': u'2015-09-21T21:25:08Z', u'uid': u'481533'}, u'pos': [39.8754277, -75.2383022], u'_id': ObjectId('592c5021ab26e56b372c57a4'), u'type': u'node', u'id': u'3753512142'}

__Comment:__
> A quick Google map search confirms that Green Leaf's is a restaurant nearby Philadelphia International Airport.

## ----------------------------------------------------------
## Comments

In working with this data, I saw a great need for fuller tagging of nodes. Many nodes are relatively incomplete: I found many nodes that were schools by name, but were not tagged as such with the amenity field. Along these lines, when auditing the data I would see addresses that were incomplete -- there would be a city name but no postcode, etc. The data would be much better if there were a minimum requirement set by OpenStreetMap when users submit nodes. For example, make the full address (house or building number, street name, city/town, state, postcode) required if it is a place, as well as at least one other tag (like "amenity" or "place" or "name").

When cleaning the data, I also noticed another way OSM could improve user submissions. I found situations where invalid information was in a field (for example, "PA" in the "postcode" field). OSM could implement simple checks to increase the chances of valid information being submitted. This would make cleaning easier and subsequent analysis more meaningful.

_Benefits:_ Address information will be more complete; nodes will have more tags completed; tag values will contain fewer to no invalid information.

_Anticipated problems:_ Contributors may skip filling in information if they do not know all of the required components; requires participation from contributors.

Another potential way to flesh out the data would be to collaborate with Uber or Lyft. These applications contain a wealth of geographic information. Not only could you use the information they already have to populate nodes, ways, and relations, but add new locales or details as well. When a customer requests a ride and gives the desired destination, you could have fields in the application where details could be entered by the customer if they don't already exist for the destination (like name, kind of amenity or place, is it BYOB, etc).

Benefits: Potential for many more people to contribute to the data; data has the opportunity to be corrected or added to; contribution will be tied to a location; earlier mentioned validity checks can be implemented.

Anticipated problems: Requires participation from contributors; contributors might not like being asked to contribute.